In [2]:
import pandas as pd
import json 
import re


### BioC Dataset

In [129]:
training_file=r"G:\GitClone\BioNER_Project\Gen2FT\new _dataset\BioC\Train.BioC.JSON"
with open(training_file, 'r') as f:
    data=json.loads(f.read())

In [62]:
data_in=data['documents'][0]['passages'][0]
data_in.keys()

dict_keys(['offset', 'text', 'annotations'])

In [63]:
print(data_in['text'])
print("Word Count:", len(data_in['text'].split()))

Hepatocyte nuclear factor-6: associations between genetic variability and type II diabetes and between genetic variability and estimates of insulin secretion.
Word Count: 20


In [70]:

for _,d in enumerate(data['documents'][0]['passages']):
    data_in=d
    
    entity=[]
    ner_tags=[]
    tags=set()

    for item in data_in['annotations']:
        tags.add(item['infons']['type'])
        ner_tags.append(item['infons']['type']+": "+item['text'])
        entity.append(item['text'])
        
    print(f"{_+1} text: ",data_in['text'],"\n\n","Unique tags: ",tags,"\n\n","NER TAGS: ",ner_tags,"\n\n\n")

1 text:  Hepatocyte nuclear factor-6: associations between genetic variability and type II diabetes and between genetic variability and estimates of insulin secretion. 

 Unique tags:  {'DiseaseOrPhenotypicFeature', 'GeneOrGeneProduct'} 

 NER TAGS:  ['GeneOrGeneProduct: Hepatocyte nuclear factor-6', 'DiseaseOrPhenotypicFeature: type II diabetes', 'GeneOrGeneProduct: insulin'] 



2 text:  The transcription factor hepatocyte nuclear factor (HNF)-6 is an upstream regulator of several genes involved in the pathogenesis of maturity-onset diabetes of the young. We therefore tested the hypothesis that variability in the HNF-6 gene is associated with subsets of Type II (non-insulin-dependent) diabetes mellitus and estimates of insulin secretion in glucose tolerant subjects.   We cloned the coding region as well as the intron-exon boundaries of the HNF-6 gene. We then examined them on genomic DNA in six MODY probands without mutations in the MODY1, MODY3 and MODY4 genes and in 54 patients wit

Not worthy to be used. Lets move to next dataset. 

### ChemNER

In [80]:
train_file=r"G:\GitClone\BioNER_Project\Gen2FT\new _dataset\ChemNER\training.csv"
df=pd.read_csv(train_file)
df.drop("PMID_Type", axis=1, inplace=True)
df.head()

,Sentence_Index,Token,Tag
0,0,We,O
1,0,implemented,O
2,0,a,O
3,0,two,O
4,0,-,O


In [128]:
df.info(), df.Tag.unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667075 entries, 0 to 667074
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Sentence_Index  667075 non-null  int64 
 1   Token           667044 non-null  object
 2   Tag             667075 non-null  object
dtypes: int64(1), object(2)
memory usage: 15.3+ MB


(None,
 array(['O', 'B-TRIVIAL', 'B-FAMILY', 'B-SYSTEMATIC', 'B-FORMULA',
        'I-SYSTEMATIC', 'I-FAMILY', 'B-ABBREVIATION', 'B-MULTIPLE',
        'I-MULTIPLE', 'I-TRIVIAL', 'B-IDENTIFIER', 'I-FORMULA',
        'I-ABBREVIATION', 'B-NO CLASS', 'I-IDENTIFIER', 'I-NO CLASS'],
       dtype=object))

In [111]:
df[(df['Sentence_Index']==5) & (df['Tag']!="O") ]

,Sentence_Index,Token,Tag
162,5,haloperidol,B-TRIVIAL
641,5,glutathione,B-TRIVIAL
646,5,superoxide,B-TRIVIAL
649,5,glutathione,B-TRIVIAL
656,5,glutathione,B-TRIVIAL
...,...,...,...
663943,5,S33084,B-IDENTIFIER
664295,5,oxytocin,B-TRIVIAL
664310,5,oxytocin,B-TRIVIAL
666768,5,oestrogen,B-FAMILY


This data is too good but still not suitable for our purpose.

### NCBI Corpus dataset

In [224]:
def splitcorpus(file):
    list_=[]
    with open(file, "+r") as f:
        temp=[]
        for line in f.readlines():
            if line.strip()!="":
                temp.append(line.strip())
            else:
                if temp!=[]:
                    list_.append(temp) 
                    temp=[]
                continue
    return {"data":list_}
# Example usage:
file_path = r'G:\GitClone\BioNER_Project\Gen2FT\NCBI Corpus\NCBItrainset_corpus.txt'
ner_data = splitcorpus(file_path)

# Output the result to a JSON file
with open(r"G:\GitClone\BioNER_Project\Gen2FT\NCBI Corpus\NCBItrainset_corpus.json", "w") as f:
    json.dump(ner_data, f, indent=4)

In [228]:
ner_tags=[]
unique_tags=set()
for _ in range(len(ner_data['data'])):
    t=ner_data['data'][_][0].split("|t|")[-1]
    a=ner_data['data'][_][1].split("|a|")[-1]
    text=t+" "+a
    temp=[]
    temp2=[]
    for i in ner_data['data'][_][2:]:
        tag=i.split('\t')[4].strip()
        entity=i.split('\t')[3].strip()
        temp.append(f'''"{tag}":"{entity}"''')
        if tag=="SpecificDisease":
            temp2.append(f'''"B-SPDIS":"{entity}"''')
        elif tag=="DiseaseClass":
            temp2.append(f'''"B-DIS":"{entity}"''')
        elif tag=="Modifier":
            temp2.append(f'''"I-MoDIS":"{entity}"''')
        elif tag=="CompositeMention":
            temp2.append(f'''"I-ComDIS":"{entity}"''')
        unique_tags.add(tag)
    ner_tags.append({"text":text,"ner_tags":list(set(temp)),"simple_tags":list(set(temp2))})
    
with open(r"G:\GitClone\BioNER_Project\Gen2FT\NCBI Corpus\NCBI_Dis_Dataset.json", "w") as f:
    json.dump(ner_tags, f)

In [221]:
unique_tags

{'CompositeMention', 'DiseaseClass', 'Modifier', 'SpecificDisease'}